In [23]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [24]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ConvNeXtXLarge
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [25]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/Data/train"
val_dir = "/content/drive/MyDrive/Colab Notebooks/Data/valid"
test_dir = "/content/drive/MyDrive/Colab Notebooks/Data/test"

In [26]:
img_height, img_width = 224, 224
batch_size = 32

In [27]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [28]:
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [29]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = "rgb"
)

Found 613 images belonging to 4 classes.


In [30]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = "rgb"
)

Found 72 images belonging to 4 classes.


In [31]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode = "rgb"
)

Found 315 images belonging to 4 classes.


In [32]:
base_model = ConvNeXtXLarge(weights = "imagenet", include_top = False, input_shape = (img_height, img_width,3))

In [33]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation = "relu")(x)
predictions = Dense(train_generator.num_classes, activation = "softmax")(x)

In [34]:
model = Model(inputs = base_model.input, outputs = predictions)

In [35]:
for layers in base_model.layers:
    layers.trainable = False

In [36]:
model.compile(optimizer = Adam(learning_rate = 0.001), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [37]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')

In [ ]:
model.fit(train_generator,
                    steps_per_epoch = train_generator.samples//batch_size,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples//batch_size,
                   epochs = 50,
                   callbacks = [checkpoint, early_stopping])